In [12]:
import pandas as pd
import math
import random
import copy
import csv

In [13]:
# ============ parameters ============
# speed: 15 km/h = 0.25 km/min
SPEED = 15 
SPEED = SPEED / 60  # 0.25 km/min
MAX_CAPACITY = 140  # Maximum cargo volume for couriers

def compute_distance(lat1, lng1, lat2, lng2):
    R = 6378.137
    lat1_rad = math.radians(lat1)
    lat2_rad = math.radians(lat2)
    lng1_rad = math.radians(lng1)
    lng2_rad = math.radians(lng2)
    dlat = lat2_rad - lat1_rad
    dlng = lng2_rad - lng1_rad
    # Haversine 
    a = (math.sin(dlat / 2) ** 2
         + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlng / 2) ** 2)
    c = 2 * math.asin(math.sqrt(a))
    distance = R * c
    return distance

def travel_time(distance_km):
    time_minutes = distance_km * 4.0
    return time_minutes

def service_time(num_packages):
    service_minutes = 3.0 * math.sqrt(num_packages) + 5.0
    return service_minutes

# ============ data structure ============
class Stop:
    def __init__(self, location_id, lat, lng, stop_type, 
                 order_id=None, packages=0,
                 earliest=0, latest=24*60,
                 paired_stop_id=None):
        """
        stop_type: "site", "delivery", or "shop"
        earliest, latest: time window constraints (in minutes from 08:00)
        paired_stop_id: for O2O, shop-stop and delivery-stop are paired
        """
        self.location_id = location_id
        self.lat = lat
        self.lng = lng
        self.stop_type = stop_type
        self.order_id = order_id
        self.packages = packages
        self.earliest = earliest  # earliest arrival time
        self.latest = latest      # latest arrival time (only for O2O delivery)
        self.arrival = None
        self.departure = None
        # If it's a shop stop, store the ID of the corresponding delivery stop, and vice versa
        self.paired_stop_id = paired_stop_id

class Route:
    def __init__(self, courier_id, start_stop):
        self.courier_id = courier_id
        self.stops = [start_stop]  # Start from site
        self.current_load = 0

    def total_time(self):
        if self.stops and self.stops[-1].departure is not None:
            return self.stops[-1].departure
        else:
            return 0
            
# ============ read data ============
def read_data():
    """
    Reads the following CSV files:
      - new_1.csv: Site data (Site_id, Lng, Lat)
      - new_2.csv: Delivery spot data (Spot_id, Lng, Lat)
      - new_3.csv: Shop data (Shop_id, Lng, Lat)
      - new_4.csv: E-commerce orders (Order_id, Spot_id, Site_id, Num)
      - new_5.csv: O2O orders (Order_id, Spot_id, Shop_id, Pickup_time, Delivery_time, Num)
      - new_6.csv: Courier list (Courier_id)
    Returns:
      A tuple of six DataFrames: (sites, spots, shops, ecommerce_orders, o2o_orders, couriers).
    """
    sites = pd.read_csv("./Dataset/new_1.csv")
    spots = pd.read_csv("./Dataset/new_2.csv")
    shops = pd.read_csv("./Dataset/new_3.csv")
    ecommerce_orders = pd.read_csv("./Dataset/new_4.csv")
    o2o_orders = pd.read_csv("./Dataset/new_5.csv")
    couriers = pd.read_csv("./Dataset/new_6.csv")
    return sites, spots, shops, ecommerce_orders, o2o_orders, couriers

# ============ Create Stop List ============
def build_stop_list(sites, spots, shops, ecommerce_orders, o2o_orders):
    """
    将电商订单 (site -> spot) 与 O2O 订单 (shop -> spot) 全部转换为 Stop 对象列表：
      - 对于电商订单，每笔订单创建两个 Stop：
          1) pickup_stop: 从网点 (site_id) 取货
          2) delivery_stop: 将包裹送至配送点 (spot_id)
      - 对于 O2O 订单，每笔订单也创建两个 Stop：
          1) shop_stop: 从商户 (shop_id) 取货
          2) delivery_stop: 将包裹送至配送点 (spot_id)
    """

    # ---------- 1) 构造网点 / 配送点 / 商户 坐标字典 ----------
    site_dict = {row['Site_id']: (row['Lat'], row['Lng']) for _, row in sites.iterrows()}
    spot_dict = {row['Spot_id']: (row['Lat'], row['Lng']) for _, row in spots.iterrows()}
    shop_dict = {row['Shop_id']: (row['Lat'], row['Lng']) for _, row in shops.iterrows()}

    all_stops = []

    # ---------- 2) 电商订单：site -> spot ----------
    # 题目要求：电商包裹在 8:00 前已到达网点，快递员须在 20:00 前送到消费者手中。
    # 因此我们简单地将 [earliest=0, latest=12*60=720] 分别赋给取货和送货节点。
    # （注意：0 表示相对 08:00 的分钟数，即 08:00 -> 0, 20:00 -> 720）
    for _, row in ecommerce_orders.iterrows():
        site_id = row['Site_id']
        spot_id = row['Spot_id']
        packages = row['Num']

        # 网点坐标
        site_lat, site_lng = site_dict[site_id]
        # 配送点坐标
        spot_lat, spot_lng = spot_dict[spot_id]

        # 创建取货节点 (pickup from site)
        pickup_stop = Stop(
            location_id = site_id,
            lat = site_lat,
            lng = site_lng,
            stop_type = "site",         # 或者你也可以用 "pickup" 表示
            order_id = row['Order_id'],
            packages = packages,
            earliest = 0,              # 8:00 开始即可取货
            latest = 12*60,            # 必须在 20:00 前取走（可自行决定是否需要更严格）
            paired_stop_id = None      # 稍后设置
        )

        # 创建送货节点 (delivery to spot)
        delivery_stop = Stop(
            location_id = spot_id,
            lat = spot_lat,
            lng = spot_lng,
            stop_type = "delivery",
            order_id = row['Order_id'],
            packages = packages,
            earliest = 0,              # 可随时送达
            latest = 12*60,            # 必须在 20:00 前送到
            paired_stop_id = None
        )

        # 互相关联，方便后续在插入或局部搜索时校验“先取后送”
        pickup_stop.paired_stop_id = delivery_stop.location_id
        delivery_stop.paired_stop_id = pickup_stop.location_id

        all_stops.append(pickup_stop)
        all_stops.append(delivery_stop)

    # ---------- 3) O2O 订单：shop -> spot ----------
    # 与电商订单类似，只是取货点变为 shop，且有明确的 Pickup_time / Delivery_time。
    def time_to_minutes(t_str):
        # 将形如 "11:00" 的时间转换为相对 08:00 的分钟数：11:00 -> (11*60 + 0) - (8*60) = 180
        hour, minute = map(int, t_str.split(":"))
        return hour * 60 + minute - 8 * 60

    for _, row in o2o_orders.iterrows():
        shop_id = row['Shop_id']
        spot_id = row['Spot_id']
        packages = row['Num']

        pickup_t = time_to_minutes(row['Pickup_time'])    # 不能早于该时间到达商户
        delivery_t = time_to_minutes(row['Delivery_time'])# 必须在该时间前送达

        # 商户坐标
        shop_lat, shop_lng = shop_dict[shop_id]
        # 配送点坐标
        spot_lat, spot_lng = spot_dict[spot_id]

        # 创建商户取货节点
        shop_stop = Stop(
            location_id = shop_id,
            lat = shop_lat,
            lng = shop_lng,
            stop_type = "shop",
            order_id = row['Order_id'],
            packages = packages,
            earliest = pickup_t,   # 不得早于 pickup_time
            latest = 12*60,        # 这里可以设置为 20:00 或更宽松
            paired_stop_id = None
        )

        # 创建送货节点
        delivery_stop = Stop(
            location_id = spot_id,
            lat = spot_lat,
            lng = spot_lng,
            stop_type = "delivery",
            order_id = row['Order_id'],
            packages = packages,
            earliest = 0,         # 可以随时送达
            latest = delivery_t,   # 必须在 delivery_time 之前送到
            paired_stop_id = None
        )

        shop_stop.paired_stop_id = delivery_stop.location_id
        delivery_stop.paired_stop_id = shop_stop.location_id

        all_stops.append(shop_stop)
        all_stops.append(delivery_stop)

    return all_stops

# ============ Initial solution construction (greedy insertion) ============
def initial_solution(sites, spots, shops, ecommerce_orders, o2o_orders, couriers):
    """
    仅以电商订单为例，采用贪心策略构造初始路线：
      1. 对每位快递员随机指定一个起始网点（取货点）。
      2. 将所有电商订单（配送点）随机排序后，依次尝试插入到各条路线末尾，
         插入时估计行驶时间和配送点服务时间，检查容量约束。
    """
    # 构造网点字典，方便根据 Site_id 获取经纬度
    site_dict = {row['Site_id']: (row['Lat'], row['Lng']) for index, row in sites.iterrows()}
    
    # 构造配送点字典
    spot_dict = {row['Spot_id']: (row['Lat'], row['Lng']) for index, row in spots.iterrows()}
    
    # 构造电商订单列表（这里只处理电商订单）
    orders = []
    for index, row in ecommerce_orders.iterrows():
        spot_id = row['Spot_id']
        lat, lng = spot_dict[spot_id]
        # 电商订单配送：取货在网点（所有订单均可在 8:00 取到），这里只需要配送到消费者对应的配送点
        order_stop = Stop(location_id=spot_id, lat=lat, lng=lng, stop_type="delivery",
                          order_id=row['Order_id'], packages=row['Num'])
        orders.append(order_stop)
    # 随机打乱顺序，模拟无序订单
    random.shuffle(orders)
    
    # 为每个快递员创建一条路线，起始点为随机选取的网点
    courier_routes = {}
    for index, row in couriers.iterrows():
        courier_id = row['Courier_id']
        site_row = sites.sample(1).iloc[0]
        lat, lng = site_row['Lat'], site_row['Lng']
        start_stop = Stop(location_id=site_row['Site_id'], lat=lat, lng=lng, stop_type="site")
        # 起始网点默认为 08:00 出发，设定到达和离开时间均为0
        start_stop.arrival = 0
        start_stop.departure = 0
        courier_routes[courier_id] = Route(courier_id, start_stop)
    
    # 贪心插入：对每个订单，选择使路线增加时间最少的插入（这里仅在末尾插入）
    for order in orders:
        best_increase = float('inf')
        best_route = None
        for route in courier_routes.values():
            last_stop = route.stops[-1]
            dist = compute_distance(last_stop.lat, last_stop.lng, order.lat, order.lng)
            t_time = travel_time(dist)
            # 预计到达时间
            est_arrival = last_stop.departure + t_time
            # 检查容量（简化：当前载货量加上该订单包裹是否超过上限）
            if route.current_load + order.packages > MAX_CAPACITY:
                continue
            # 计算配送点的服务时间
            s_time = service_time(order.packages)
            new_departure = est_arrival + s_time
            increase = new_departure - route.total_time()
            if increase < best_increase:
                best_increase = increase
                best_route = route
        if best_route is not None:
            # 将订单插入到最佳路线的末尾
            last_stop = best_route.stops[-1]
            dist = compute_distance(last_stop.lat, last_stop.lng, order.lat, order.lng)
            t_time = travel_time(dist)
            arrival_time = last_stop.departure + t_time
            s_time = service_time(order.packages)
            departure_time = arrival_time + s_time
            order.arrival = arrival_time
            order.departure = departure_time
            best_route.stops.append(order)
            best_route.current_load += order.packages
    return courier_routes

# ============ 路径时间重计算 ============
def recalc_route_times(route):
    """
    根据路线中各站点之间的行驶时间和服务时间（配送点），重新计算到达和离开时间。
    这里简单处理，对于网点和商户（pickup）可设为0服务时间，配送点按公式计算。
    """
    for i, stop in enumerate(route.stops):
        if i == 0:
            # 起始点
            stop.arrival = 0
            stop.departure = 0
            continue
        prev_stop = route.stops[i-1]
        dist = compute_distance(prev_stop.lat, prev_stop.lng, stop.lat, stop.lng)
        t_time = travel_time(dist)
        arrival_time = prev_stop.departure + t_time
        # 如果有时间窗要求，可在此调整（如等待至 earliest）
        if arrival_time < stop.earliest:
            arrival_time = stop.earliest
        if stop.stop_type == "delivery":
            s_time = service_time(stop.packages)
        else:
            s_time = 0
        departure_time = arrival_time + s_time
        stop.arrival = arrival_time
        stop.departure = departure_time
    return route.stops[-1].departure  # 返回路线总耗时

# ============ 局部搜索：2-opt 交换 ============
def two_opt(route):
    """
    对单个路线应用 2-opt 交换，遍历所有可能的交换对，若交换后总耗时下降，则接受新路线。
    注意：实际场景需同时验证所有约束，这里仅以总时间作为优化目标。
    """
    best_route = copy.deepcopy(route)
    best_total_time = recalc_route_times(best_route)
    improved = True
    while improved:
        improved = False
        n = len(best_route.stops)
        for i in range(1, n - 2):
            for j in range(i + 1, n - 1):
                new_route = copy.deepcopy(best_route)
                # 反转 i 到 j 之间的顺序
                new_route.stops[i:j+1] = list(reversed(new_route.stops[i:j+1]))
                total_time = recalc_route_times(new_route)
                if total_time < best_total_time:
                    best_route = new_route
                    best_total_time = total_time
                    improved = True
        # 如果本次循环有改进，则继续寻找更优解
    return best_route

def local_search(courier_routes):
    """
    对每个快递员路线执行局部搜索（2-opt）
    """
    improved_routes = {}
    for courier_id, route in courier_routes.items():
        new_route = two_opt(route)
        improved_routes[courier_id] = new_route
    return improved_routes

# ============ save to CSV ===============
def save_schedule_to_csv(improved_routes, filename="submission.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Courier_id", "Addr", "Arrival_time", "Departure", "Amount", "Order_id"])

        for courier_id, route in improved_routes.items():
            # 可根据需要对 stops 按到达时间排序，确保顺序正确
            route.stops.sort(key=lambda s: (s.arrival if s.arrival is not None else math.inf))

            for stop in route.stops:
                # 根据 stop_type 判断取/送货量
                if stop.stop_type in ("site", "shop"):
                    # 网点或商户 -> 取货
                    amount = stop.packages
                else:
                    # "delivery" -> 送货
                    amount = -stop.packages

                # 将 None 时间处理为 0 或根据需要处理
                arrival_time = int(round(stop.arrival)) if stop.arrival is not None else 0
                departure_time = int(round(stop.departure)) if stop.departure is not None else 0

                writer.writerow([
                    courier_id,         # Courier_id
                    stop.location_id,   # Addr
                    arrival_time,       # Arrival_time
                    departure_time,     # Departure_time
                    amount,             # Amount
                    stop.order_id       # Order_id
                ])

    print(f"Schedule successfully saved to {filename}")

In [14]:
# ============ 主函数 ============
def main():
    sites, spots, shops, ecommerce_orders, o2o_orders, couriers = read_data()
    print("数据读取完成。")
    # 构造初始解
    courier_routes = initial_solution(sites, spots, shops, ecommerce_orders, o2o_orders, couriers)
    print("初始解构造完成。")
    # 局部搜索优化
    improved_routes = local_search(courier_routes)
    print("局部搜索完成。")

    # 4. 计算并输出“总时间”或“最大时间”
    #    （也可结合服务时间、惩罚等，构造完整的目标函数）
    total_time = 0
    max_time = 0
    for courier_id, route in improved_routes.items():
        # 重新计算路线，以得到最后离开时间
        route_time = recalc_route_times(route)  
        total_time += route_time
        if route_time > max_time:
            max_time = route_time
    print(f"所有快递员路线的【总时间】为: {total_time} 分钟")
    print(f"所有快递员路线的【最大时间】为: {max_time} 分钟")
    
    # 5. 保存调度计划
    save_schedule_to_csv(improved_routes, "submission.csv")
    print("调度计划已保存至 submission.csv")

    # 6. 可视化或打印具体路线
    for courier_id, route in improved_routes.items():
        print(f"\n快递员 {courier_id} 路径：")
        for stop in route.stops:
            print(f"  站点: {stop.location_id} ({stop.stop_type}), "
                  f"到达: {stop.arrival}, 离开: {stop.departure}, "
                  f"订单: {stop.order_id}, 包裹数: {stop.packages}")
    
if __name__ == "__main__":
    main()

数据读取完成。
初始解构造完成。
局部搜索完成。
所有快递员路线的【总时间】为: 200490.8759138169 分钟
所有快递员路线的【最大时间】为: 757.6331059033522 分钟
Schedule successfully saved to submission.csv
调度计划已保存至 submission.csv

快递员 D0001 路径：
  站点: A097 (site), 到达: 0, 离开: 0, 订单: None, 包裹数: 0
  站点: B7982 (delivery), 到达: 10.681506708045104, 离开: 32.113183433200085, 订单: F7264, 包裹数: 30
  站点: B3000 (delivery), 到达: 34.474229389173075, 离开: 55.90590611432806, 订单: F7225, 包裹数: 30
  站点: B2581 (delivery), 到达: 58.59388772509207, 离开: 74.8188598854139, 订单: F7217, 包裹数: 14
  站点: B0263 (delivery), 到达: 81.44494785608224, 离开: 104.93818986498917, 订单: F7202, 包裹数: 38
  站点: B6752 (delivery), 到达: 109.5328728220419, 离开: 121.24107675454127, 订单: F7254, 包裹数: 5
  站点: B9055 (delivery), 到达: 123.54055666105687, 离开: 137.54055666105688, 订单: F7274, 包裹数: 9
  站点: B6889 (delivery), 到达: 141.4423227066261, 离开: 156.83462755203936, 订单: F7257, 包裹数: 12
  站点: B5408 (delivery), 到达: 275.4731074180687, 离开: 284.71574810518797, 订单: F2808, 包裹数: 2

快递员 D0002 路径：
  站点: A052 (site), 到达: 0, 离开: 0, 